In [1]:
# import warnings
# warnings.filterwarnings("ignore")

import requests
import nba_api
from nba_api.stats.endpoints import teamdashboardbygeneralsplits, leaguedashteamstats

from datetime import date, timedelta
import time

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.decomposition import PCA
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from xgboost import XGBClassifier


# from sklearn import model_selection
# from sklearn.utils import class_weight
# from sklearn.metrics import confusion_matrix

In [ ]:
# Compile a dictionary of all 30 NBA teams and their Team ID
teams = {"Atlanta Hawks": 1610612737,
        "Boston Celtics": 1610612738,
        "Brooklyn Nets": 1610612751,
        "Charlotte Bobcats": 1610612766,
        "Charlotte Hornets": 1610612766,
        "Chicago Bulls": 1610612741,
        "Cleveland Cavaliers": 1610612739,
        "Dallas Mavericks": 1610612742,
        "Denver Nuggets": 1610612743,
        "Detroit Pistons": 1610612765,
        "Golden State Warriors": 1610612744,
        "Houston Rockets": 1610612745,
        "Indiana Pacers": 1610612754,
        "LA Clippers": 1610612746,
        "Los Angeles Clippers": 1610612746,
        "Los Angeles Lakers": 1610612747,
        "Memphis Grizzlies": 1610612763,
        "Miami Heat": 1610612748,
        "Milwaukee Bucks": 1610612749,
        "Minnesota Timberwolves": 1610612750,
        "New Jersey Nets": 1610612751,
        "New Orleans Hornets": 1610612740,
        "New Orleans Pelicans": 1610612740,
        "New York Knicks": 1610612752,
        "Oklahoma City Thunder": 1610612760,
        "Orlando Magic": 1610612753,
        "Philadelphia 76ers": 1610612755,
        "Phoenix Suns": 1610612756,
        "Portland Trail Blazers": 1610612757,
        "Sacramento Kings": 1610612758,
        "San Antonio Spurs": 1610612759,
        "Toronto Raptors": 1610612761,
        "Utah Jazz": 1610612762,
        "Washington Wizards": 1610612764,
    }

In [ ]:
# Compile a dictionary of stats and their source 
available_stats = {'W_PCT': 'Base',
                   'FG_PCT': 'Base',
                   'FG3_PCT': 'Base',
                   'FT_PCT': 'Base',
                   'REB': 'Base',
                   'AST': 'Base',
                   'TOV': 'Base',
                   'STL': 'Base',
                   'BLK': 'Base',
                   'PLUS_MINUS': 'Base',
                   'OFF_RATING': 'Advanced',
                   'DEF_RATING': 'Advanced',
                   'TS_PCT': 'Advanced'}

In [ ]:
# get_stats.py gets the team data for the model

# from team_names import teams


def get_team_stats_dict(team, start_date, end_date, season='2021-22'):
    """
    Returns the stats for the selected team in a dataframe, default year is 2021-22
    :param start_data: Day of games scheduled in form 'mm/dd/yyyy'
    :param end_data: Day of games scheduled in form 'mm/dd/yyyy'
    :param season: Day of games scheduled in form 'yyyy-yy'
    :return: A dictionary of game matchups {home_team:[away_team]}
    """

    time.sleep(1)

    
    general_team_info = teamdashboardbygeneralsplits.TeamDashboardByGeneralSplits(team_id=teams[team],
                                                                                  per_mode_detailed='Per100Possessions',
                                                                                  season=season,
                                                                                  date_from_nullable=start_date,
                                                                                  date_to_nullable=end_date,
                                                                                  timeout=120)
    general_team_dict = general_team_info.get_normalized_dict()
    general_team_dashboard = general_team_dict['OverallTeamDashboard'][0]


    win_percentage = general_team_dashboard['W_PCT']
    fg_percentage = general_team_dashboard['FG_PCT']
    fg3_percentage = general_team_dashboard['FG3_PCT']
    ft_percentage = general_team_dashboard['FT_PCT']
    rebounds = general_team_dashboard['REB']
    assists = general_team_dashboard['AST']
    turnovers = general_team_dashboard['TOV']
    steals = general_team_dashboard['STL']
    blocks = general_team_dashboard['BLK']
    plus_minus = general_team_dashboard['PLUS_MINUS']

    advanced_team_info = teamdashboardbygeneralsplits.TeamDashboardByGeneralSplits(team_id=teams[team],
                                                                                   measure_type_detailed_defense='Advanced',
                                                                                   season=season,
                                                                                   date_from_nullable=start_date,
                                                                                   date_to_nullable=end_date,
                                                                                   timeout=120)
    advanced_team_dict = advanced_team_info.get_normalized_dict()
    advanced_team_dashboard = advanced_team_dict['OverallTeamDashboard'][0]

    offensive_rating = advanced_team_dashboard['OFF_RATING']
    defensive_rating = advanced_team_dashboard['DEF_RATING']
    true_shooting_percentage = advanced_team_dashboard['TS_PCT']

    all_stats_dict = {
        'W_PCT': win_percentage,
        'FG_PCT': fg_percentage,
        'FG3_PCT': fg3_percentage,
        'FT_PCT': ft_percentage,
        'REB': rebounds,
        'AST': assists,
        'TOV': turnovers,
        'STL': steals,
        'BLK': blocks,
        'PLUS_MINUS': plus_minus,
        'OFF_RATING': offensive_rating,
        'DEF_RATING': defensive_rating,
        'TS_PCT': true_shooting_percentage
    }

    return all_stats_dict


get_team_stats_dict('Golden State Warriors', '10/19/2021', '04/10/2022', '2021-22')

In [ ]:
# get_matches.py gets the daily matches for a specific date and the results of the games

from nba_api.stats.endpoints import leaguegamelog, scoreboard, leaguestandings
#from team_names import teams



def get_match_results(date, season):
    """
    Returns the matchup and result of the game

    :param date: Day of games scheduled in form 'mm/dd/yyyy'
    :param season: Season in form of 'yyyy-yy'
    :return: [{Golden State Warriors: Boston Celtics}], ['W']
    """

    game_log = leaguegamelog.LeagueGameLog(season=season, league_id='00', date_from_nullable=date,
                                           date_to_nullable=date, season_type_all_star='Regular Season', timeout=120)
    game_log_dict = game_log.get_normalized_dict()
    list_of_teams = game_log_dict['LeagueGameLog']

    daily_match = {}
    win_loss = []
    score = []
    game_id = []

    for i in range(0, len(list_of_teams), 2):

        if '@' in list_of_teams[i]['MATCHUP']:

            away_team = list_of_teams[i]['TEAM_NAME']
            home_team = list_of_teams[i + 1]['TEAM_NAME']

            win_loss.append(list_of_teams[i + 1]['WL'])

            game_id.append(list_of_teams[i + 1]['GAME_ID'])

            score.append(list_of_teams[i + 1]['PTS'])
            score.append(list_of_teams[i]['PTS'])

        else:
            away_team = list_of_teams[i + 1]['TEAM_NAME']
            home_team = list_of_teams[i]['TEAM_NAME']

            win_loss.append(list_of_teams[i]['WL'])

            game_id.append(list_of_teams[i]['GAME_ID'])

            score.append(list_of_teams[i]['PTS'])
            score.append(list_of_teams[i + 1]['PTS'])

        daily_match.update({home_team: away_team})

    match_results = [daily_match, win_loss, score, game_id]

    return match_results


def get_daily_matches(date):
    """
    This method creates a dictionary of daily game matchups.

    :param date: Day of games scheduled in form 'mm/dd/yyyy'
    :return: A dictionary of game matchups {home_team:away_team}
    """

    daily_match = scoreboard.Scoreboard(league_id='00', game_date=date, timeout=120)
    daily_match_dict = daily_match.get_normalized_dict()
    games = daily_match_dict['GameHeader']

    match = {}

    for game in games:

        home_team_id = game['HOME_TEAM_ID']

        for team, team_id in teams.items():
            if team_id == home_team_id:
                home_team = team

        away_team_id = game['VISITOR_TEAM_ID']

        for team, team_id in teams.items():
            if team_id == away_team_id:
                away_team = team

        match.update({home_team: away_team})

    return match



def main():
    print(f"""'get_daily_matches' returns a dictionary of the games on a specified date\n{get_daily_matches('12/25/22')}\n""")
    print(f"""'get_match_results' returns the matchup plus the result\n{get_match_results('10/19/2021', '2021-22')}""")

main()

In [ ]:
from datetime import date, timedelta
import pandas as pd
import requests

# from get_stats import get_team_stats_dict
# from get_matches import get_match_results
#from standardization import z_score, stat_std, stat_mean
#from available_stats import available_stats


# [{'Sacramento Kings': 'Boston Celtics', 'Charlotte Hornets': 'Philadelphia 76ers'}, ['W', 'L']]
# team stats is a dataframe
def to_dataframe(daily_games, start_date, end_date, season): #, mean_dict, std_dict):
    full_dataframe = []
    game_number = 0  # counter to match with the correct game
    daily_results = daily_games[1]  # win or loss for each game
    score = daily_games[2]
    game_id = daily_games[3]

    for home_team, away_team in daily_games[0].items():  # loops through matchups
        home_team_stats = get_team_stats_dict(home_team, start_date, end_date, season)
        away_team_stats = get_team_stats_dict(away_team, start_date, end_date, season)

        current_game = [home_team, away_team]
        
        current_game.append(game_id[game_number])

        current_game.append(score.pop(0))

        for stat, stat_type in available_stats.items():
            current_game.append(home_team_stats[stat])
        
        current_game.append(score.pop(0))

        for stat, stat_type in available_stats.items():
            current_game.append(away_team_stats[stat])


        #for stat, stat_type in available_stats.items():
        #    z_score_diff = z_score_difference(home_team_stats[stat], away_team_stats[stat], mean_dict[stat], std_dict[stat])

         #   current_game.append(z_score_diff)

        if daily_results[game_number] == 'W':
            result = 1
        else:
            result = 0

        current_game.append(result)
        game_number += 1

        print(current_game)

        full_dataframe.append(current_game)

    return full_dataframe



def date_range(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)
        

def training_set(start_year, start_month, start_day, end_year, end_month, end_day, season, season_start):
    start_date = date(start_year, start_month, start_day)
    end_date = date(end_year, end_month, end_day)

    total_games = []

    for single_date in date_range(start_date, end_date):
        current_date = single_date.strftime('%m/%d/%Y')
        print(current_date)

        previous_day = single_date - timedelta(days=1)
        previous_day_formatted = previous_day.strftime('%m/%d/%Y')

        #mean_std_dictionary = mean_std_dict(season_start, previous_day_formatted, season)
        #mean_dict = mean_std_dictionary[0]
        #std_dict = mean_std_dictionary[1]

        current_day_games = get_match_results(current_date, season)
        current_day_games_with_stats = to_dataframe(current_day_games, season_start, previous_day_formatted, season)

        for game in current_day_games_with_stats:
            game.append(current_date)
            total_games.append(game)

    print(total_games)
    return total_games


def make_dataframe(game_list):
    games = pd.DataFrame(game_list,
                         columns=['Home', 'Away', 'Game_ID', 'H_Score', 'H_W_PCT', 'H_FG_PCT', 'H_FG3_PCT', 'H_FT_PCT',
                                  'H_REB', 'H_AST', 'H_TOV', 'H_STL',
                                  'H_BLK', 'H_PLUS_MINUS', 'H_OFF_RATING', 'H_DEF_RATING', 'H_TS_PCT', 'A_Score',
                                  'A_W_PCT', 'A_FG_PCT', 'A_FG3_PCT',
                                  'A_FT_PCT', 'A_REB', 'A_AST', 'A_TOV', 'A_STL',
                                  'A_BLK', 'A_PLUS_MINUS', 'A_OFF_RATING', 'A_DEF_RATING', 'A_TS_PCT', 'Result',
                                  'Date'])

    print(games)
    return games


def main():
    attempts = 10

    for i in range(attempts):
        try:
            all_games = training_set(start_year=2017, start_month=10, start_day=17, end_year=2018, end_month=4, end_day=12,
                             season='2017-18', season_start='10/17/2017')
            df = make_dataframe(all_games)

            print(df)
            df.to_csv(r'C:\Users\alvaro\OneDrive\Documents\School\Flatiron\Projects\NBA_Prediction_Model\data\nba_df_2017.csv', index=False)
        except requests.exceptions.ReadTimeout:
            if i < attempts - 1:
                continue
            else:
                raise
        break

if __name__ == '__main__':
    main()


# Data

## 2017 - 18

In [2]:
df_2017 = pd.read_csv('./data/nba_df_2017.csv')
df_2017['Date'] = pd.to_datetime(df_2017['Date'])
df_2017['Season'] = '2017-18'

df_2017_2 = pd.read_csv('./data/nba_df_2017_v2.csv')
df_2017_2['Date'] = pd.to_datetime(df_2017_2['Date'])
df_2017_2['Season'] = '2017-18'

df_2017_3 = pd.read_csv('./data/nba_df_2017_v3.csv')
df_2017_3['Date'] = pd.to_datetime(df_2017_3['Date'])
df_2017_3['Season'] = '2017-18'

print(len(df_2017), len(df_2017_2), len(df_2017_3))

171 546 493


In [3]:
frames = [df_2017, df_2017_2, df_2017_3]
df_2017_final = pd.concat(frames)
len(df_2017_final)

1210

## 2018 - 19

In [4]:
df_2018 = pd.read_csv('./data/nba_df_2018.csv')
df_2018['Date'] = pd.to_datetime(df_2018['Date'])
df_2018['Season'] = '2018-19'

print(len(df_2018))

1203


## 2019 - 20

In [5]:
df_2019 = pd.read_csv('./data/nba_df_2019.csv')
df_2019['Date'] = pd.to_datetime(df_2019['Date'])
df_2019['Season'] = '2019-20'

df_2019_2 = pd.read_csv('./data/nba_df_2019_2.csv')
df_2019_2['Date'] = pd.to_datetime(df_2019_2['Date'])
df_2019_2['Season'] = '2019-20'

print(len(df_2019), len(df_2019_2))

951 84


In [6]:
frames = [df_2019, df_2019_2]
df_2019_final = pd.concat(frames)
len(df_2019_final)

1035

## 2020 - 21

In [7]:
df_2021 = pd.read_csv('./data/nba_df_2020_v0.csv')
df_2021['Date'] = pd.to_datetime(df_2021['Date'])
df_2021['Season'] = '2020-21'

df_2021_2 = pd.read_csv('./data/nba_df_2020.csv')
df_2021_2['Date'] = pd.to_datetime(df_2021_2['Date'])
df_2021_2['Season'] = '2020-21'

df_2021_3 = pd.read_csv('./data/nba_df_2020_v2.csv')
df_2021_3['Date'] = pd.to_datetime(df_2021_3['Date'])
df_2021_3['Season'] = '2020-21'

print(len(df_2021), len(df_2021_2), len(df_2021_3))

5 582 469


In [8]:
frames = [df_2021, df_2021_2, df_2021_3]
df_2021_final = pd.concat(frames)
len(df_2021_final)

1056

## 2021 - 22

In [9]:
import pandas as pd

df_2022 = pd.read_csv('./data/nba_game_2022.csv')
df_2022['Date'] = pd.to_datetime(df_2022['Date'])
df_2022['Season'] = '2021-22'

df_2022_1 = pd.read_csv('./data/nba_game_2022_v1.csv')
df_2022_1['Date'] = pd.to_datetime(df_2022_1['Date'])
df_2022_1['Season'] = '2021-22'

df_2022_2 = pd.read_csv('./data/nba_game_2022_v2.csv')
df_2022_2['Date'] = pd.to_datetime(df_2022_2['Date'])
df_2022_2['Season'] = '2021-22'

df_2022_3 = pd.read_csv('./data/nba_game_2022_v3.csv')
df_2022_3['Date'] = pd.to_datetime(df_2022_3['Date'])
df_2022_3['Season'] = '2021-22'

print(len(df_2022), len(df_2022_1), len(df_2022_2), len(df_2022_3))

61 159 784 199


In [10]:
frames = [df_2022, df_2022_1, df_2022_2, df_2022_3]
df_2022_final = pd.concat(frames)

print(f"Length of 2022 data: {len(df_2022_final)}\n")

Length of 2022 data: 1203



## Merge DataFrames

In [11]:
frames = [df_2017_final, df_2018, df_2019_final, df_2021_final, df_2022_final]
df = pd.concat(frames)
df = df.reset_index(drop=True)

df

,Home,Away,Game_ID,H_Score,H_W_PCT,H_FG_PCT,H_FG3_PCT,H_FT_PCT,H_REB,H_AST,...,A_TOV,A_STL,A_BLK,A_PLUS_MINUS,A_OFF_RATING,A_DEF_RATING,A_TS_PCT,Result,Date,Season
0,Philadelphia 76ers,Boston Celtics,21700019,92,0.000,0.462,0.429,0.737,45.3,23.6,...,13.5,11.5,3.0,-5.5,99.5,106.1,0.499,0,2017-10-20,2017-18
1,Phoenix Suns,Los Angeles Lakers,21700026,130,0.000,0.315,0.259,0.722,33.0,10.0,...,19.0,7.6,6.7,-15.2,87.6,101.9,0.455,0,2017-10-20,2017-18
2,Brooklyn Nets,Orlando Magic,21700022,126,0.000,0.479,0.400,0.906,37.4,19.1,...,14.4,7.7,8.7,6.7,111.5,103.8,0.564,1,2017-10-20,2017-18
3,Milwaukee Bucks,Cleveland Cavaliers,21700021,97,1.000,0.500,0.333,0.833,45.0,19.0,...,17.3,3.1,4.1,3.1,104.1,100.0,0.543,0,2017-10-20,2017-18
4,Indiana Pacers,Portland Trail Blazers,21700018,96,1.000,0.520,0.265,0.781,40.2,24.8,...,17.8,7.9,6.9,47.5,122.8,76.0,0.606,0,2017-10-20,2017-18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5702,Orlando Magic,Miami Heat,22101227,125,0.259,0.433,0.329,0.786,44.1,23.6,...,15.0,7.6,3.3,4.8,113.0,108.2,0.584,1,2022-04-10,2021-22
5703,Denver Nuggets,Los Angeles Lakers,22101220,141,0.593,0.482,0.353,0.796,44.6,28.0,...,14.3,7.5,5.1,-3.1,109.7,112.7,0.566,0,2022-04-10,2021-22
5704,Memphis Grizzlies,Boston Celtics,22101223,110,0.691,0.462,0.353,0.735,48.7,25.7,...,13.8,7.4,6.0,7.1,113.3,106.2,0.577,0,2022-04-10,2021-22
5705,Philadelphia 76ers,Detroit Pistons,22101228,118,0.617,0.465,0.366,0.820,43.5,24.4,...,14.2,7.8,4.8,-7.7,105.5,113.2,0.533,1,2022-04-10,2021-22


In [12]:
df.isnull().sum()

Home            0
Away            0
Game_ID         0
H_Score         0
H_W_PCT         0
H_FG_PCT        0
H_FG3_PCT       0
H_FT_PCT        0
H_REB           0
H_AST           0
H_TOV           0
H_STL           0
H_BLK           0
H_PLUS_MINUS    0
H_OFF_RATING    0
H_DEF_RATING    0
H_TS_PCT        0
A_Score         0
A_W_PCT         0
A_FG_PCT        0
A_FG3_PCT       0
A_FT_PCT        0
A_REB           0
A_AST           0
A_TOV           0
A_STL           0
A_BLK           0
A_PLUS_MINUS    0
A_OFF_RATING    0
A_DEF_RATING    0
A_TS_PCT        0
Result          0
Date            0
Season          0
dtype: int64

In [13]:
duplicates = df[df.duplicated()]
duplicates

,Home,Away,Game_ID,H_Score,H_W_PCT,H_FG_PCT,H_FG3_PCT,H_FT_PCT,H_REB,H_AST,...,A_TOV,A_STL,A_BLK,A_PLUS_MINUS,A_OFF_RATING,A_DEF_RATING,A_TS_PCT,Result,Date,Season


# Last N Games Win %

In [14]:
import warnings
warnings.filterwarnings("ignore")
prev_game_df = df[df['Date'] < '12/25/2022'][(df['Home'] == "Golden State Warriors") | (df['Away'] == 'Golden State Warriors')].sort_values(by = 'Date').tail(10)
prev_game_df
h_df = prev_game_df.iloc[:, range(0, 32, 31)]

h_df = h_df.loc[h_df['Home'] == 'Golden State Warriors'] 
print(h_df)

                       Home  Result
5621  Golden State Warriors       0
5641  Golden State Warriors       1
5677  Golden State Warriors       1


In [15]:
def get_avg_win_pct_last_n_games(team, game_date, df, n):
    prev_game_df = df[df['Date'] < game_date][(df['Home'] == team) | (df['Away'] == team)].sort_values(by = 'Date').tail(n)
    
    wins = 0 
    
    result_df = prev_game_df.iloc[:, range(0,32,31)]
    h_df = result_df.loc[result_df['Home'] == team] 
    
    h_wins = h_df.loc[h_df['Result'] == 1]
    
    wins += len(h_wins)
      
    a_df = result_df.loc[result_df['Home'] != team]
    a_wins = a_df.loc[a_df['Result'] == 0]
    
    wins += len(a_wins)

    return wins/n
get_avg_win_pct_last_n_games('Golden State Warriors', '12/25/2022', df, 10)

0.6

In [16]:
for season in df['Season'].unique() :
    
    season_stats = df[df['Season'] == season].sort_values(by='Date').reset_index(drop=True)
    
    for index, row in df.iterrows() : 
        game_id = row['Game_ID']
        game_date = row['Date']
        h_team = row['Home']
        a_team = row['Away']
        
        df.loc[index,'Home_W_Pct_10'] = get_avg_win_pct_last_n_games(h_team, game_date, df, 10)
        
        df.loc[index,'Away_W_Pct_10'] = get_avg_win_pct_last_n_games(a_team, game_date, df, 10)
        

In [17]:
df[df['Season'] == '2021-22'].tail()


,Home,Away,Game_ID,H_Score,H_W_PCT,H_FG_PCT,H_FG3_PCT,H_FT_PCT,H_REB,H_AST,...,A_BLK,A_PLUS_MINUS,A_OFF_RATING,A_DEF_RATING,A_TS_PCT,Result,Date,Season,Home_W_Pct_10,Away_W_Pct_10
5702,Orlando Magic,Miami Heat,22101227,125,0.259,0.433,0.329,0.786,44.1,23.6,...,3.3,4.8,113.0,108.2,0.584,1,2022-04-10,2021-22,0.3,0.6
5703,Denver Nuggets,Los Angeles Lakers,22101220,141,0.593,0.482,0.353,0.796,44.6,28.0,...,5.1,-3.1,109.7,112.7,0.566,0,2022-04-10,2021-22,0.6,0.2
5704,Memphis Grizzlies,Boston Celtics,22101223,110,0.691,0.462,0.353,0.735,48.7,25.7,...,6.0,7.1,113.3,106.2,0.577,0,2022-04-10,2021-22,0.8,0.7
5705,Philadelphia 76ers,Detroit Pistons,22101228,118,0.617,0.465,0.366,0.820,43.5,24.4,...,4.8,-7.7,105.5,113.2,0.533,1,2022-04-10,2021-22,0.6,0.4
5706,Minnesota Timberwolves,Chicago Bulls,22101224,120,0.568,0.457,0.358,0.778,43.5,25.2,...,4.2,-0.4,112.6,113.1,0.578,0,2022-04-10,2021-22,0.5,0.3


In [ ]:
df

# ELO Rating
- every team starts with a 1500
$$R_{i+1} = k * (S_{team} - E_{team} + R_{i})$$
- S team is 1 if the team wins and 0 if they lose
- E team is the expected win probability of the team 
$$E_{team} = \frac{1}{1+10^{\frac{opp\_elo - team\_elo}{400}}}$$
- k is a moving constant that depends on margin of victory and difference in Elo ratings
$$k = 20\frac{(MOV_{winner} + 3)^{0.8}}{7.5 + 0.006(elo\_difference_{winner})} $$
- team year by year carryover 
$$(R * 0.75) + (0.25 * 1505)$$

In [18]:
df.head()

,Home,Away,Game_ID,H_Score,H_W_PCT,H_FG_PCT,H_FG3_PCT,H_FT_PCT,H_REB,H_AST,...,A_BLK,A_PLUS_MINUS,A_OFF_RATING,A_DEF_RATING,A_TS_PCT,Result,Date,Season,Home_W_Pct_10,Away_W_Pct_10
0,Philadelphia 76ers,Boston Celtics,21700019,92,0.0,0.462,0.429,0.737,45.3,23.6,...,3.0,-5.5,99.5,106.1,0.499,0,2017-10-20,2017-18,0.0,0.0
1,Phoenix Suns,Los Angeles Lakers,21700026,130,0.0,0.315,0.259,0.722,33.0,10.0,...,6.7,-15.2,87.6,101.9,0.455,0,2017-10-20,2017-18,0.0,0.0
2,Brooklyn Nets,Orlando Magic,21700022,126,0.0,0.479,0.400,0.906,37.4,19.1,...,8.7,6.7,111.5,103.8,0.564,1,2017-10-20,2017-18,0.0,0.0
3,Milwaukee Bucks,Cleveland Cavaliers,21700021,97,1.0,0.500,0.333,0.833,45.0,19.0,...,4.1,3.1,104.1,100.0,0.543,0,2017-10-20,2017-18,0.0,0.0
4,Indiana Pacers,Portland Trail Blazers,21700018,96,1.0,0.520,0.265,0.781,40.2,24.8,...,6.9,47.5,122.8,76.0,0.606,0,2017-10-20,2017-18,0.0,0.0


In [19]:
# Home and road team win probabilities implied by Elo ratings and home court adjustment 
import math
import time
def win_probs(home_elo, away_elo, home_court_advantage) :
    h = math.pow(10, home_elo/400)
    r = math.pow(10, away_elo/400)
    a = math.pow(10, home_court_advantage/400) 

    denom = r + a*h
    home_prob = a*h / denom
    away_prob = r / denom 
  
    return home_prob, away_prob

  #odds the home team will win based on elo ratings and home court advantage

def home_odds_on(home_elo, away_elo, home_court_advantage) :
    h = math.pow(10, home_elo/400)
    r = math.pow(10, away_elo/400)
    a = math.pow(10, home_court_advantage/400)
    return a*h/r

#this function determines the constant used in the elo rating, based on margin of victory and difference in elo ratings
def elo_k(MOV, elo_diff):
    k = 20 # Optimal K is 20 https://fivethirtyeight.com/features/how-we-calculate-nba-elo-ratings/
    if MOV>0:
        multiplier=(MOV+3)**(0.8)/(7.5+0.006*(elo_diff))
    else:
        multiplier=(-MOV+3)**(0.8)/(7.5+0.006*(-elo_diff))
    return k*multiplier


# Updates the home and away teams elo ratings after a game 

def update_elo(home_score, away_score, home_elo, away_elo, home_court_advantage) :
    home_prob, away_prob = win_probs(home_elo, away_elo, home_court_advantage) 

    if (home_score - away_score > 0) :
        home_win = 1 
        away_win = 0 
    else :
        home_win = 0 
        away_win = 1 
  
    k = elo_k(home_score - away_score, home_elo - away_elo)

    updated_home_elo = home_elo + k * (home_win - home_prob) 
    updated_away_elo = away_elo + k * (away_win - away_prob)
    
    return updated_home_elo, updated_away_elo


# Takes into account prev season elo
# The reason we revert to a mean of 1505 rather than 1500 is that 
# there are liable to be a couple of relatively recent expansion teams in the league at any given time
def get_prev_elo(team, date, season, team_stats, elo_df) :
    prev_game = team_stats[team_stats['Date'] < game_date][(team_stats['Home'] == team) | (team_stats['Away'] == team)].sort_values(by = 'Date').tail(1).iloc[0] 

    if team == prev_game['Home'] :
        elo_rating = elo_df[elo_df['Game_ID'] == prev_game['Game_ID']]['H_Team_Elo_After'].values[0]
    else :
        elo_rating = elo_df[elo_df['Game_ID'] == prev_game['Game_ID']]['A_Team_Elo_After'].values[0]
  
    if prev_game['Season'] != season :
        return (0.75 * elo_rating) + (0.25 * 1505) # Year-to-Year Carry-Over
    else :
        return elo_rating

In [20]:
df.sort_values(by = 'Date', inplace = True)
df.reset_index(inplace=True, drop = True)
elo_df = pd.DataFrame(columns=['Game_ID', 'H_Team', 'A_Team', 'H_Team_Elo_Before', 'A_Team_Elo_Before', 'H_Team_Elo_After', 'A_Team_Elo_After'])
teams_elo_df = pd.DataFrame(columns=['Game_ID','Team', 'Elo', 'Date', 'Where_Played', 'Season']) 

for index, row in df.iterrows(): 
    game_id = row['Game_ID']
    game_date = row['Date']
    season = row['Season']
    h_team, a_team = row['Home'], row['Away']
    h_score, a_score = row['H_Score'], row['A_Score'] 

    if (h_team not in elo_df['H_Team'].values and h_team not in elo_df['A_Team'].values) :
        h_team_elo_before = 1500
    else :
        h_team_elo_before = get_prev_elo(h_team, game_date, season, df, elo_df)

    if (a_team not in elo_df['H_Team'].values and a_team not in elo_df['A_Team'].values) :
        a_team_elo_before = 1500
    else :
        a_team_elo_before = get_prev_elo(a_team, game_date, season, df, elo_df)

    h_team_elo_after, a_team_elo_after = update_elo(h_score, a_score, h_team_elo_before, a_team_elo_before, 69)

    new_row = {'Game_ID': game_id, 'H_Team': h_team, 'A_Team': a_team, 'H_Team_Elo_Before': h_team_elo_before, 'A_Team_Elo_Before': a_team_elo_before, \
                                                                        'H_Team_Elo_After' : h_team_elo_after, 'A_Team_Elo_After': a_team_elo_after}
    teams_row_one = {'Game_ID': game_id,'Team': h_team, 'Elo': h_team_elo_before, 'Date': game_date, 'Where_Played': 'Home', 'Season': season}
    teams_row_two = {'Game_ID': game_id,'Team': a_team, 'Elo': a_team_elo_before, 'Date': game_date, 'Where_Played': 'Away', 'Season': season}
  
    elo_df = elo_df.append(new_row, ignore_index = True)
    teams_elo_df = teams_elo_df.append(teams_row_one, ignore_index=True)
    teams_elo_df = teams_elo_df.append(teams_row_two, ignore_index=True)

In [21]:
#teams_elo_df.set_index(["Team"], append=True)
#dataset = teams_elo_df.pivot(index="Team",values="Elo", columns="Date")
dates = list(set([d.strftime("%m-%d-%Y") for d in teams_elo_df["Date"]]))
dates = sorted(dates, key=lambda x: time.strptime(x, '%m-%d-%Y'))
teams = df["Away"]
dataset = pd.DataFrame(columns=dates)
dataset["Team"] = teams.drop_duplicates()
dataset = dataset.set_index("Team")

for index, row in teams_elo_df.iterrows():
    date = row["Date"].strftime("%m-%d-%Y")
    team = row["Team"]
    elo = row["Elo"]
    dataset[date][team] = elo

teams_elo_df['Elo'] = teams_elo_df['Elo'].astype(float)

elo_df

,Game_ID,H_Team,A_Team,H_Team_Elo_Before,A_Team_Elo_Before,H_Team_Elo_After,A_Team_Elo_After
0,21700019,Philadelphia 76ers,Boston Celtics,1500,1500,1487.588205,1512.411795
1,21700026,Phoenix Suns,Los Angeles Lakers,1500,1500,1494.220957,1505.779043
2,21700022,Brooklyn Nets,Orlando Magic,1500,1500,1505.657856,1494.342144
3,21700021,Milwaukee Bucks,Cleveland Cavaliers,1500,1500,1481.093199,1518.906801
4,21700018,Indiana Pacers,Portland Trail Blazers,1500,1500,1481.783902,1518.216098
...,...,...,...,...,...,...,...
5702,22101225,New Orleans Pelicans,Golden State Warriors,1534.445418,1574.302989,1516.647138,1592.101269
5703,22101219,Dallas Mavericks,San Antonio Spurs,1585.484564,1490.992295,1590.900483,1485.576375
5704,22101222,LA Clippers,Oklahoma City Thunder,1526.825496,1338.090516,1537.08683,1327.829182
5705,22101217,Charlotte Hornets,Washington Wizards,1539.195491,1428.163212,1545.957475,1421.401228


In [ ]:
df.head()

In [22]:
df = df.merge(elo_df.drop(columns=['H_Team', 'A_Team']), on ='Game_ID')
df.head()

,Home,Away,Game_ID,H_Score,H_W_PCT,H_FG_PCT,H_FG3_PCT,H_FT_PCT,H_REB,H_AST,...,A_TS_PCT,Result,Date,Season,Home_W_Pct_10,Away_W_Pct_10,H_Team_Elo_Before,A_Team_Elo_Before,H_Team_Elo_After,A_Team_Elo_After
0,Philadelphia 76ers,Boston Celtics,21700019,92,0.0,0.462,0.429,0.737,45.3,23.6,...,0.499,0,2017-10-20,2017-18,0.0,0.0,1500,1500,1487.588205,1512.411795
1,Phoenix Suns,Los Angeles Lakers,21700026,130,0.0,0.315,0.259,0.722,33.0,10.0,...,0.455,0,2017-10-20,2017-18,0.0,0.0,1500,1500,1494.220957,1505.779043
2,Brooklyn Nets,Orlando Magic,21700022,126,0.0,0.479,0.400,0.906,37.4,19.1,...,0.564,1,2017-10-20,2017-18,0.0,0.0,1500,1500,1505.657856,1494.342144
3,Milwaukee Bucks,Cleveland Cavaliers,21700021,97,1.0,0.500,0.333,0.833,45.0,19.0,...,0.543,0,2017-10-20,2017-18,0.0,0.0,1500,1500,1481.093199,1518.906801
4,Indiana Pacers,Portland Trail Blazers,21700018,96,1.0,0.520,0.265,0.781,40.2,24.8,...,0.606,0,2017-10-20,2017-18,0.0,0.0,1500,1500,1481.783902,1518.216098


# Merging Dataset

# Standardization and Z Score

In [ ]:
df.head()

# Evaluate Different Models - No Z Score

In [23]:
df.to_csv(r'C:/Users/alvaro/OneDrive/Documents/School/Flatiron/Projects/NBA_Prediction_Model/data/nba_raw.csv', 
          index=False)
print(f'The final dataset consists of three seasons and {len(df)} games.')
df = df.reset_index(drop=True)
df.tail()


The final dataset consists of three seasons and 5707 games.


,Home,Away,Game_ID,H_Score,H_W_PCT,H_FG_PCT,H_FG3_PCT,H_FT_PCT,H_REB,H_AST,...,A_TS_PCT,Result,Date,Season,Home_W_Pct_10,Away_W_Pct_10,H_Team_Elo_Before,A_Team_Elo_Before,H_Team_Elo_After,A_Team_Elo_After
5702,New Orleans Pelicans,Golden State Warriors,22101225,107,0.444,0.457,0.332,0.789,46.0,25.4,...,0.580,0,2022-04-10,2021-22,0.6,0.5,1534.445418,1574.302989,1516.647138,1592.101269
5703,Dallas Mavericks,San Antonio Spurs,22101219,130,0.630,0.460,0.348,0.771,44.8,24.2,...,0.556,1,2022-04-10,2021-22,0.8,0.7,1585.484564,1490.992295,1590.900483,1485.576375
5704,LA Clippers,Oklahoma City Thunder,22101222,138,0.506,0.457,0.372,0.792,44.3,24.1,...,0.531,1,2022-04-10,2021-22,0.5,0.4,1526.825496,1338.090516,1537.08683,1327.829182
5705,Charlotte Hornets,Washington Wizards,22101217,124,0.519,0.467,0.363,0.741,43.9,27.6,...,0.569,1,2022-04-10,2021-22,0.6,0.5,1539.195491,1428.163212,1545.957475,1421.401228
5706,Minnesota Timberwolves,Chicago Bulls,22101224,120,0.568,0.457,0.358,0.778,43.5,25.2,...,0.578,0,2022-04-10,2021-22,0.5,0.3,1574.46694,1450.007328,1563.891946,1460.582322


In [24]:
df = pd.read_csv('./data/nba_raw.csv')

In [25]:
print(df.corr()['Result'].abs().sort_values(ascending=False))

Result               1.000000
H_Score              0.461871
A_Score              0.461385
H_Team_Elo_After     0.363042
A_Team_Elo_After     0.308683
H_Team_Elo_Before    0.266572
H_PLUS_MINUS         0.234252
Home_W_Pct_10        0.215551
H_W_PCT              0.214225
A_Team_Elo_Before    0.209484
A_W_PCT              0.169454
A_PLUS_MINUS         0.169382
H_OFF_RATING         0.157338
Away_W_Pct_10        0.157198
H_TS_PCT             0.154047
H_DEF_RATING         0.152833
H_FG_PCT             0.140374
A_OFF_RATING         0.136559
A_TS_PCT             0.129107
H_REB                0.114393
A_FG_PCT             0.101940
H_FG3_PCT            0.090817
A_DEF_RATING         0.089300
H_AST                0.080855
A_REB                0.076496
A_AST                0.073782
H_BLK                0.072406
A_BLK                0.066711
H_FT_PCT             0.064787
A_FG3_PCT            0.061637
A_TOV                0.054370
A_FT_PCT             0.051991
H_STL                0.041594
H_TOV     

In [ ]:
with sns.axes_style("darkgrid"):
    fig, ax = plt.subplots(figsize=(44, 34))
    
    mask = np.triu(np.ones_like(df.corr(), dtype=bool))
    ax = sns.heatmap(abs(df.corr()),mask=mask,annot=True)
    fig.savefig('images/Corelation_Heatmap');

In [26]:
# Cast "H_Team_Elo_Before" and "A_Team_Elo_Before" as floats
df["H_Team_Elo_Before"] = df.H_Team_Elo_Before.astype(float)
df["A_Team_Elo_Before"] = df.A_Team_Elo_Before.astype(float)

In [27]:
# Remove columns that have data on post-game stats
df = df.drop(['H_Team_Elo_After', 'A_Team_Elo_After', 'H_Score', 'A_Score'], axis=1)

In [28]:
# Remove irrelevant columns
df = df.drop(['Home', 'Away', 'Game_ID', 'Date', 'Season'], axis=1)

In [29]:
df.columns

Index(['H_W_PCT', 'H_FG_PCT', 'H_FG3_PCT', 'H_FT_PCT', 'H_REB', 'H_AST',
       'H_TOV', 'H_STL', 'H_BLK', 'H_PLUS_MINUS', 'H_OFF_RATING',
       'H_DEF_RATING', 'H_TS_PCT', 'A_W_PCT', 'A_FG_PCT', 'A_FG3_PCT',
       'A_FT_PCT', 'A_REB', 'A_AST', 'A_TOV', 'A_STL', 'A_BLK', 'A_PLUS_MINUS',
       'A_OFF_RATING', 'A_DEF_RATING', 'A_TS_PCT', 'Result', 'Home_W_Pct_10',
       'Away_W_Pct_10', 'H_Team_Elo_Before', 'A_Team_Elo_Before'],
      dtype='object')

In [30]:
df.to_csv(r'C:\Users\alvaro\OneDrive\Documents\School\Flatiron\Projects\NBA_Prediction_Model\data\nba.csv', 
          index=False)

In [31]:

df = pd.read_csv('./data/nba.csv')

In [ ]:
df

# EDA

In [ ]:
plt.figure(figsize=(44, 34))
correlation = df[['H_W_PCT', 
                  'H_REB', 
                  'H_AST',
                  'H_TOV', 
                  'H_STL', 
                  'H_BLK', 
                  'H_PLUS_MINUS', 
                  'H_OFF_RATING',
                  'H_DEF_RATING', 
                  'H_TS_PCT', 
                  'H_Team_Elo_Before', 
                  'Home_W_Pct_10', 
                  'Result'
                  ]]

sns.heatmap(correlation.corr(), annot=True);
# correlation
# sns.heatmap(df.corr(), annot=True);
# sns.heatmap(df['Result'].corr(), annot=True);

In [ ]:
print(df.corr()['Result'].abs().sort_values(ascending=False))

In [ ]:
with sns.axes_style("darkgrid"):
    fig, ax = plt.subplots(figsize=(44, 34))
    
    mask = np.triu(np.ones_like(df.corr(), dtype=bool))
    ax = sns.heatmap(abs(df.corr()),mask=mask,annot=True)
    fig.savefig('images/Corelation_Heatmap_2');

In [ ]:
df

In [32]:
X = df.drop(columns = 'Result')

y = df['Result']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

print(f'X train shape: {X_train.shape}')
print(f'X test shape: {X_test.shape}')

X train shape: (3823, 30)
X test shape: (1884, 30)


In [ ]:
print(f"""Raw Counts 
{df["Result"].value_counts()}\n
Percentages 
{df["Result"].value_counts(normalize=True)}


We would get an accuracy score of {np.round(df["Result"].value_counts(normalize=True)[1], 4)} with a baseline model, i.e. about 56.6% accuracy

This is because about 55.79% of the results are wins""")

In [ ]:
print(f"""Train percent wins
{y_train.value_counts(normalize=True)}\n""")

print(f"""Test percent wins: 
{y_test.value_counts(normalize=True)}\n""")

In [ ]:
class ModelWithCV():
    '''Structure to save the model and more easily see its crossvalidation'''
    
    def __init__(self, model, model_name, X, y, cv_now=True):
        self.model = model
        self.name = model_name
        self.X = X
        self.y = y
        # For CV results
        self.cv_results = None
        self.cv_mean = None
        self.cv_median = None
        self.cv_std = None
        #
        if cv_now:
            self.cross_validate()
        
    def cross_validate(self, X=None, y=None, kfolds=10):
        '''
        Perform cross-validation and return results.
        
        Args: 
          X:
            Optional; Training data to perform CV on. Otherwise use X from object
          y:
            Optional; Training data to perform CV on. Otherwise use y from object
          kfolds:
            Optional; Number of folds for CV (default is 10)  
        '''
        
        cv_X = X if X else self.X
        cv_y = y if y else self.y

        self.cv_results = cross_val_score(self.model, cv_X, cv_y, cv=kfolds)
        self.cv_mean = np.mean(self.cv_results)
        self.cv_median = np.median(self.cv_results)
        self.cv_std = np.std(self.cv_results)

        
    def print_cv_summary(self):
        cv_summary = (
        f'''CV Results for `{self.name}` model:
            {self.cv_mean:.5f} ± {self.cv_std:.5f} accuracy
        ''')
        print(cv_summary)

        
    def plot_cv(self, ax):
        '''
        Plot the cross-validation values using the array of results and given 
        Axis for plotting.
        '''
        ax.set_title(f'CV Results for `{self.name}` Model')
        # Thinner violinplot with higher bw
        sns.violinplot(y=self.cv_results, ax=ax, bw=.4)
        sns.swarmplot(
                y=self.cv_results,
                color='orange',
                size=10,
                alpha= 0.8,
                ax=ax
        )

        return ax

## Dummy

In [ ]:
# Create Dummy/Baseliner



estimator = DummyRegressor(strategy='mean')

# Create Dummy/Baseliner
from sklearn.dummy import DummyRegressor

pipe = Pipeline(steps=[
    ('estimator', DummyRegressor(strategy='mean'))
])

cv = ModelWithCV(pipe, 'estimator', X_train, y_train)

In [ ]:
f, ax = plt.subplots()

cv.plot_cv(ax);

In [ ]:
cv.print_cv_summary()

## Logistic Regression

In [33]:
pipe = Pipeline(steps=[
    #('scaler', StandardScaler()),
    #('pca', PCA(n_components=4)),
    ('estimator', LogisticRegression(random_state=42))
])

In [34]:
param_grid = {}
param_grid['estimator__C'] = np.logspace(-4, 4, 50)
param_grid['estimator__solver'] = ['saga']
param_grid['estimator__penalty'] = ['elasticnet']
#param_grid['estimator__class_weight'] = ['balanced', None]
param_grid['estimator__n_jobs'] = [-1]
param_grid['estimator__l1_ratio'] = [0, 1]

In [35]:
grid_search = GridSearchCV(pipe, 
                           param_grid, 
                           cv=10, 
                           return_train_score=True, 
                           scoring='accuracy', 
                           verbose=2)

In [36]:
grid_search.fit(X_train, y_train)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits
[CV] END estimator__C=0.0001, estimator__l1_ratio=0, estimator__n_jobs=-1, estimator__penalty=elasticnet, estimator__solver=saga; total time=   2.3s
[CV] END estimator__C=0.0001, estimator__l1_ratio=0, estimator__n_jobs=-1, estimator__penalty=elasticnet, estimator__solver=saga; total time=   0.3s
[CV] END estimator__C=0.0001, estimator__l1_ratio=0, estimator__n_jobs=-1, estimator__penalty=elasticnet, estimator__solver=saga; total time=   0.3s
[CV] END estimator__C=0.0001, estimator__l1_ratio=0, estimator__n_jobs=-1, estimator__penalty=elasticnet, estimator__solver=saga; total time=   0.3s
[CV] END estimator__C=0.0001, estimator__l1_ratio=0, estimator__n_jobs=-1, estimator__penalty=elasticnet, estimator__solver=saga; total time=   0.3s
[CV] END estimator__C=0.0001, estimator__l1_ratio=0, estimator__n_jobs=-1, estimator__penalty=elasticnet, estimator__solver=saga; total time=   0.4s
[CV] END estimator__C=0.0001, estimator__

[CV] END estimator__C=0.00021209508879201905, estimator__l1_ratio=1, estimator__n_jobs=-1, estimator__penalty=elasticnet, estimator__solver=saga; total time=   0.0s
[CV] END estimator__C=0.00021209508879201905, estimator__l1_ratio=1, estimator__n_jobs=-1, estimator__penalty=elasticnet, estimator__solver=saga; total time=   0.0s
[CV] END estimator__C=0.00021209508879201905, estimator__l1_ratio=1, estimator__n_jobs=-1, estimator__penalty=elasticnet, estimator__solver=saga; total time=   0.0s
[CV] END estimator__C=0.00030888435964774815, estimator__l1_ratio=0, estimator__n_jobs=-1, estimator__penalty=elasticnet, estimator__solver=saga; total time=   0.0s
[CV] END estimator__C=0.00030888435964774815, estimator__l1_ratio=0, estimator__n_jobs=-1, estimator__penalty=elasticnet, estimator__solver=saga; total time=   0.0s
[CV] END estimator__C=0.00030888435964774815, estimator__l1_ratio=0, estimator__n_jobs=-1, estimator__penalty=elasticnet, estimator__solver=saga; total time=   0.0s
[CV] END e

[CV] END estimator__C=0.0006551285568595509, estimator__l1_ratio=0, estimator__n_jobs=-1, estimator__penalty=elasticnet, estimator__solver=saga; total time=   0.0s
[CV] END estimator__C=0.0006551285568595509, estimator__l1_ratio=0, estimator__n_jobs=-1, estimator__penalty=elasticnet, estimator__solver=saga; total time=   0.0s
[CV] END estimator__C=0.0006551285568595509, estimator__l1_ratio=0, estimator__n_jobs=-1, estimator__penalty=elasticnet, estimator__solver=saga; total time=   0.0s
[CV] END estimator__C=0.0006551285568595509, estimator__l1_ratio=1, estimator__n_jobs=-1, estimator__penalty=elasticnet, estimator__solver=saga; total time=   0.0s
[CV] END estimator__C=0.0006551285568595509, estimator__l1_ratio=1, estimator__n_jobs=-1, estimator__penalty=elasticnet, estimator__solver=saga; total time=   0.0s
[CV] END estimator__C=0.0006551285568595509, estimator__l1_ratio=1, estimator__n_jobs=-1, estimator__penalty=elasticnet, estimator__solver=saga; total time=   0.0s
[CV] END estimat

[CV] END estimator__C=0.0013894954943731374, estimator__l1_ratio=1, estimator__n_jobs=-1, estimator__penalty=elasticnet, estimator__solver=saga; total time=   0.0s
[CV] END estimator__C=0.0013894954943731374, estimator__l1_ratio=1, estimator__n_jobs=-1, estimator__penalty=elasticnet, estimator__solver=saga; total time=   0.0s
[CV] END estimator__C=0.0013894954943731374, estimator__l1_ratio=1, estimator__n_jobs=-1, estimator__penalty=elasticnet, estimator__solver=saga; total time=   0.0s
[CV] END estimator__C=0.0020235896477251557, estimator__l1_ratio=0, estimator__n_jobs=-1, estimator__penalty=elasticnet, estimator__solver=saga; total time=   0.0s
[CV] END estimator__C=0.0020235896477251557, estimator__l1_ratio=0, estimator__n_jobs=-1, estimator__penalty=elasticnet, estimator__solver=saga; total time=   0.0s
[CV] END estimator__C=0.0020235896477251557, estimator__l1_ratio=0, estimator__n_jobs=-1, estimator__penalty=elasticnet, estimator__solver=saga; total time=   0.0s
[CV] END estimat

[CV] END estimator__C=0.004291934260128779, estimator__l1_ratio=0, estimator__n_jobs=-1, estimator__penalty=elasticnet, estimator__solver=saga; total time=   0.0s
[CV] END estimator__C=0.004291934260128779, estimator__l1_ratio=0, estimator__n_jobs=-1, estimator__penalty=elasticnet, estimator__solver=saga; total time=   0.0s
[CV] END estimator__C=0.004291934260128779, estimator__l1_ratio=1, estimator__n_jobs=-1, estimator__penalty=elasticnet, estimator__solver=saga; total time=   0.0s
[CV] END estimator__C=0.004291934260128779, estimator__l1_ratio=1, estimator__n_jobs=-1, estimator__penalty=elasticnet, estimator__solver=saga; total time=   0.0s
[CV] END estimator__C=0.004291934260128779, estimator__l1_ratio=1, estimator__n_jobs=-1, estimator__penalty=elasticnet, estimator__solver=saga; total time=   0.0s
[CV] END estimator__C=0.004291934260128779, estimator__l1_ratio=1, estimator__n_jobs=-1, estimator__penalty=elasticnet, estimator__solver=saga; total time=   0.0s
[CV] END estimator__C=

...

[CV] END estimator__C=494.1713361323828, estimator__l1_ratio=0, estimator__n_jobs=-1, estimator__penalty=elasticnet, estimator__solver=saga; total time=   0.0s
[CV] END estimator__C=494.1713361323828, estimator__l1_ratio=1, estimator__n_jobs=-1, estimator__penalty=elasticnet, estimator__solver=saga; total time=   0.1s
[CV] END estimator__C=494.1713361323828, estimator__l1_ratio=1, estimator__n_jobs=-1, estimator__penalty=elasticnet, estimator__solver=saga; total time=   0.0s
[CV] END estimator__C=494.1713361323828, estimator__l1_ratio=1, estimator__n_jobs=-1, estimator__penalty=elasticnet, estimator__solver=saga; total time=   0.0s
[CV] END estimator__C=494.1713361323828, estimator__l1_ratio=1, estimator__n_jobs=-1, estimator__penalty=elasticnet, estimator__solver=saga; total time=   0.0s
[CV] END estimator__C=494.1713361323828, estimator__l1_ratio=1, estimator__n_jobs=-1, estimator__penalty=elasticnet, estimator__solver=saga; total time=   0.0s
[CV] END estimator__C=494.1713361323828,

[CV] END estimator__C=1526.4179671752302, estimator__l1_ratio=0, estimator__n_jobs=-1, estimator__penalty=elasticnet, estimator__solver=saga; total time=   0.0s
[CV] END estimator__C=1526.4179671752302, estimator__l1_ratio=0, estimator__n_jobs=-1, estimator__penalty=elasticnet, estimator__solver=saga; total time=   0.0s
[CV] END estimator__C=1526.4179671752302, estimator__l1_ratio=0, estimator__n_jobs=-1, estimator__penalty=elasticnet, estimator__solver=saga; total time=   0.0s
[CV] END estimator__C=1526.4179671752302, estimator__l1_ratio=0, estimator__n_jobs=-1, estimator__penalty=elasticnet, estimator__solver=saga; total time=   0.0s
[CV] END estimator__C=1526.4179671752302, estimator__l1_ratio=0, estimator__n_jobs=-1, estimator__penalty=elasticnet, estimator__solver=saga; total time=   0.0s
[CV] END estimator__C=1526.4179671752302, estimator__l1_ratio=0, estimator__n_jobs=-1, estimator__penalty=elasticnet, estimator__solver=saga; total time=   0.0s
[CV] END estimator__C=1526.4179671

[CV] END estimator__C=3237.45754281764, estimator__l1_ratio=1, estimator__n_jobs=-1, estimator__penalty=elasticnet, estimator__solver=saga; total time=   0.0s
[CV] END estimator__C=3237.45754281764, estimator__l1_ratio=1, estimator__n_jobs=-1, estimator__penalty=elasticnet, estimator__solver=saga; total time=   0.0s
[CV] END estimator__C=3237.45754281764, estimator__l1_ratio=1, estimator__n_jobs=-1, estimator__penalty=elasticnet, estimator__solver=saga; total time=   0.0s
[CV] END estimator__C=3237.45754281764, estimator__l1_ratio=1, estimator__n_jobs=-1, estimator__penalty=elasticnet, estimator__solver=saga; total time=   0.0s
[CV] END estimator__C=4714.8663634573895, estimator__l1_ratio=0, estimator__n_jobs=-1, estimator__penalty=elasticnet, estimator__solver=saga; total time=   0.0s
[CV] END estimator__C=4714.8663634573895, estimator__l1_ratio=0, estimator__n_jobs=-1, estimator__penalty=elasticnet, estimator__solver=saga; total time=   0.0s
[CV] END estimator__C=4714.8663634573895, 

[CV] END estimator__C=10000.0, estimator__l1_ratio=0, estimator__n_jobs=-1, estimator__penalty=elasticnet, estimator__solver=saga; total time=   0.0s
[CV] END estimator__C=10000.0, estimator__l1_ratio=1, estimator__n_jobs=-1, estimator__penalty=elasticnet, estimator__solver=saga; total time=   0.0s
[CV] END estimator__C=10000.0, estimator__l1_ratio=1, estimator__n_jobs=-1, estimator__penalty=elasticnet, estimator__solver=saga; total time=   0.0s
[CV] END estimator__C=10000.0, estimator__l1_ratio=1, estimator__n_jobs=-1, estimator__penalty=elasticnet, estimator__solver=saga; total time=   0.0s
[CV] END estimator__C=10000.0, estimator__l1_ratio=1, estimator__n_jobs=-1, estimator__penalty=elasticnet, estimator__solver=saga; total time=   0.0s
[CV] END estimator__C=10000.0, estimator__l1_ratio=1, estimator__n_jobs=-1, estimator__penalty=elasticnet, estimator__solver=saga; total time=   0.0s
[CV] END estimator__C=10000.0, estimator__l1_ratio=1, estimator__n_jobs=-1, estimator__penalty=elast

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('estimator',
                                        LogisticRegression(random_state=42))]),
             param_grid={'estimator__C': array([1.00000000e-04, 1.45634848e-04, 2.12095089e-04, 3.08884360e-04,
       4.49843267e-04, 6.55128557e-04, 9.54095476e-04, 1.38949549e-03,
       2.02358965e-03, 2.94705170e-03, 4.29193426e-03, 6.25055193e-03,
       9.10298178e-03, 1.32571137e-02, 1.9306977...
       7.54312006e+01, 1.09854114e+02, 1.59985872e+02, 2.32995181e+02,
       3.39322177e+02, 4.94171336e+02, 7.19685673e+02, 1.04811313e+03,
       1.52641797e+03, 2.22299648e+03, 3.23745754e+03, 4.71486636e+03,
       6.86648845e+03, 1.00000000e+04]),
                         'estimator__l1_ratio': [0, 1],
                         'estimator__n_jobs': [-1],
                         'estimator__penalty': ['elasticnet'],
                         'estimator__solver': ['saga']},
             return_train_score=True, scoring='accuracy', verbose=2)

In [37]:
# Mean training score
grid_train_score = np.mean(grid_search.cv_results_['mean_train_score'])

# Mean test score
grid_test_score = grid_search.score(X_test, y_test)

best_grid = grid_search.best_estimator_
best_grid.fit(X_train, y_train)
y_pred = best_grid.predict(X_test)

print(f"Mean Training Score: {grid_train_score:.2%}\n")
print(f"Mean Test Score: {grid_test_score:.2%}\n")

print(f"Accuracy Score: {accuracy_score(y_test, y_pred):.2%}\n")
print(f"Optimal Parameters: {grid_search.best_params_}\n")
print(f"Testing Accuracy: {grid_search.best_score_:.2%}\n")

print(classification_report(y_test, y_pred))

Mean Training Score: 64.34%

Mean Test Score: 65.34%

Accuracy Score: 65.34%

Optimal Parameters: {'estimator__C': 0.019306977288832496, 'estimator__l1_ratio': 1, 'estimator__n_jobs': -1, 'estimator__penalty': 'elasticnet', 'estimator__solver': 'saga'}

Testing Accuracy: 64.24%

              precision    recall  f1-score   support

           0       0.66      0.47      0.55       840
           1       0.65      0.80      0.72      1044

    accuracy                           0.65      1884
   macro avg       0.65      0.64      0.63      1884
weighted avg       0.65      0.65      0.64      1884



## Random Forest - Random Search CV

In [38]:
pipe = Pipeline(steps=[
    ('estimator', RandomForestClassifier(random_state=42))
])

In [39]:
param_grid = {}
param_grid['estimator__n_estimators'] = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
param_grid['estimator__max_features'] = ['auto', 'sqrt']
param_grid['estimator__max_depth'] = [int(x) for x in np.linspace(10, 110, num = 11)]
param_grid['estimator__min_samples_split'] = [2, 5, 10]
param_grid['estimator__min_samples_leaf'] = [1, 2, 4]
param_grid['estimator__bootstrap'] = [True, False]


In [40]:
from sklearn.model_selection import RandomizedSearchCV

grid_search = RandomizedSearchCV(estimator=pipe, 
                                 param_distributions=param_grid, 
                                 cv=10, return_train_score=True, 
                                 scoring='accuracy', 
                                 n_iter=100, 
                                 random_state=42, 
                                 n_jobs=-1, 
                                 verbose=2)


In [41]:
grid_search.fit(X_train, y_train)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


KeyboardInterrupt: 

In [ ]:
# Mean training score
grid_train_score = np.mean(grid_search.cv_results_['mean_train_score'])

# Mean test score
grid_test_score = grid_search.score(X_test, y_test)

best_grid = grid_search.best_estimator_
best_grid.fit(X_train, y_train)
y_pred = best_grid.predict(X_test)

print(f"Mean Training Score: {grid_train_score:.2%}\n")
print(f"Mean Test Score: {grid_test_score:.2%}\n")

print(f"Accuracy Score: {accuracy_score(y_test, y_pred):.2%}\n")
print(f"Optimal Parameters: {grid_search.best_params_}\n")
print(f"Testing Accuracy: {grid_search.best_score_:.2%}\n")

print(classification_report(y_test, y_pred))

## Random Forest - Grid Search CV

In [ ]:
pipe = Pipeline(steps=[
    ('estimator', RandomForestClassifier(random_state=42))
])

In [ ]:
param_grid = {}
param_grid['estimator__n_estimators'] = [int(x) for x in np.linspace(start=200, stop=2000, num=10)]
param_grid['estimator__max_features'] = ['auto', 'sqrt']
param_grid['estimator__max_depth'] = [int(x) for x in np.linspace(10, 110, num=11)]
param_grid['estimator__min_samples_split'] = [2, 5, 10]
param_grid['estimator__min_samples_leaf'] = [1, 2, 4]
param_grid['estimator__bootstrap'] = [True, False]


In [ ]:
grid_search = GridSearchCV(estimator=pipe, 
                           param_grid=param_grid, 
                           cv=10, 
                           return_train_score=True, 
                           scoring='accuracy',
                           n_jobs=-1,
                           verbose=1)


In [ ]:
grid_search.fit(X_train, y_train)

In [ ]:
# Mean training score
grid_train_score = np.mean(grid_search.cv_results_['mean_train_score'])

# Mean test score
grid_test_score = grid_search.score(X_test, y_test)

best_grid = grid_search.best_estimator_
best_grid.fit(X_train, y_train)
y_pred = best_grid.predict(X_test)

print(f"Mean Training Score: {grid_train_score:.2%}\n")
print(f"Mean Test Score: {grid_test_score:.2%}\n")

print(f"Accuracy Score: {accuracy_score(y_test, y_pred):.2%}\n")
print(f"Optimal Parameters: {grid_search.best_params_}\n")
print(f"Testing Accuracy: {grid_search.best_score_:.2%}\n")

print(classification_report(y_test, y_pred))

## Gaussian Naive Bayes

In [ ]:
pipe = Pipeline(steps=[
    ('estimator', GaussianNB())
])

In [ ]:
param_grid = {}
param_grid['estimator__var_smoothing'] = np.logspace(0,-11, num=100)
#param_grid['estimator__var_smoothing'] = [1e-11, 1e-10, 1e-9]


In [ ]:
grid_search = RandomizedSearchCV(estimator=pipe, 
                                 param_distributions=param_grid, 
                                 cv=10, 
                                 return_train_score=True, 
                                 scoring='accuracy', 
                                 random_state=42,
                                 n_jobs = -1,
                                 verbose=2)


In [ ]:
grid_search.fit(X_train, y_train)

In [ ]:
# Mean training score
grid_train_score = np.mean(grid_search.cv_results_['mean_train_score'])

# Mean test score
grid_test_score = grid_search.score(X_test, y_test)

best_grid = grid_search.best_estimator_
best_grid.fit(X_train, y_train)
y_pred = best_grid.predict(X_test)

print(f"Mean Training Score: {grid_train_score:.2%}\n")
print(f"Mean Test Score: {grid_test_score:.2%}\n")

print(f"Accuracy Score: {accuracy_score(y_test, y_pred):.2%}\n")
print(f"Optimal Parameters: {grid_search.best_params_}\n")
print(f"Testing Accuracy: {grid_search.best_score_:.2%}\n")

print(classification_report(y_test, y_pred))

In [ ]:
pipe = Pipeline(steps=[
    ('estimator', GaussianNB())
])

In [ ]:
param_grid = {}
param_grid['estimator__var_smoothing'] = np.logspace(0,-11, num=100)


In [ ]:
grid_search = GridSearchCV(estimator=pipe, 
                                 param_distributions=param_grid, 
                                 cv=10, 
                                 return_train_score=True, 
                                 scoring='accuracy', 
                                 random_state=42,
                                 n_jobs = -1,
                                 verbose=2)


In [ ]:
grid_search.fit(X_train, y_train)

In [ ]:
# Mean training score
grid_train_score = np.mean(grid_search.cv_results_['mean_train_score'])

# Mean test score
grid_test_score = grid_search.score(X_test, y_test)

best_grid = grid_search.best_estimator_
best_grid.fit(X_train, y_train)
y_pred = best_grid.predict(X_test)

print(f"Mean Training Score: {grid_train_score:.2%}\n")
print(f"Mean Test Score: {grid_test_score:.2%}\n")

print(f"Accuracy Score: {accuracy_score(y_test, y_pred):.2%}\n")
print(f"Optimal Parameters: {grid_search.best_params_}\n")
print(f"Testing Accuracy: {grid_search.best_score_:.2%}\n")

print(classification_report(y_test, y_pred))

## K Nearest Neighbors

In [ ]:
pipe = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('estimator', KNeighborsClassifier())
])KNeighborsClassifierKNeighborsClassifier

In [ ]:
param_grid = {}
param_grid['estimator__n_neighbors'] = (1, 10, 1)
param_grid['estimator__leaf_size'] = (20, 40, 1)
param_grid['estimator__p'] = (1, 2)
param_grid['estimator__weights'] = ('uniform', 'distance')
param_grid['estimator__metric'] = ('minkowski', 'chebyshev')



In [ ]:
grid_search = GridSearchCV(pipe, 
                           param_grid, 
                           cv=10, 
                           return_train_score=True, 
                           scoring='accuracy', 
                           verbose=2)

In [ ]:
grid_search.fit(X_train, y_train)

In [ ]:
# Mean training score
grid_train_score = np.mean(grid_search.cv_results_['mean_train_score'])

# Mean test score
grid_test_score = grid_search.score(X_test, y_test)

best_grid = grid_search.best_estimator_
best_grid.fit(X_train, y_train)
y_pred = best_grid.predict(X_test)

print(f"Mean Training Score: {grid_train_score:.2%}\n")
print(f"Mean Test Score: {grid_test_score:.2%}\n")

print(f"Accuracy Score: {accuracy_score(y_test, y_pred):.2%}\n")
print(f"Optimal Parameters: {grid_search.best_params_}\n")
print(f"Testing Accuracy: {grid_search.best_score_:.2%}\n")

print(classification_report(y_test, y_pred))

## XGB

In [ ]:
pipe = Pipeline(steps=[
    ('estimator', xgboost.XGBRegressor(random_state=42, objective='reg:squarederror'))
])



In [ ]:
param_grid = {}
param_grid['estimator__min_child_weight'] = [1, 5, 10],
param_grid['estimator__gamma'] = [0.5, 1, 1.5, 2, 5],
param_grid['estimator__subsample'] = [0.6, 0.8, 1.0],
param_grid['estimator__colsample_bytree'] = [0.6, 0.8, 1.0],
param_grid['estimator__max_depth'] = [3, 4, 5]

In [ ]:
grid_search = GridSearchCV(estimator=pipe, 
                           param_grid=param_grid, 
                           cv=10, 
                           return_train_score=True, 
                           scoring='accuracy',
                           n_jobs = -1,
                           verbose=1)


In [ ]:
grid_search.fit(X_train, y_train)

In [ ]:
# Mean training score
grid_train_score = np.mean(grid_search.cv_results_['mean_train_score'])

# Mean test score
grid_test_score = grid_search.score(X_test, y_test)

best_grid = grid_search.best_estimator_
best_grid.fit(X_train, y_train)
y_pred = best_grid.predict(X_test)

print(f"Mean Training Score: {grid_train_score:.2%}\n")
print(f"Mean Test Score: {grid_test_score:.2%}\n")

print(f"Accuracy Score: {accuracy_score(y_test, y_pred):.2%}\n")
print(f"Optimal Parameters: {grid_search.best_params_}\n")
print(f"Testing Accuracy: {grid_search.best_score_:.2%}\n")

print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn import model_selection
from sklearn.utils import class_weight
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import numpy as np
import pandas as pd


#script to test the effectivenes of each model, uses default parameters
#test six different classification models 
def run_exps(X_train, y_train, X_test, y_test) :
    '''
    Lightweight script to test many models and find winners
    :param X_train: training split
    :param y_train: training target vector
    :param X_test: test split
    :param y_test: test target vector
    :return: DataFrame of predictions
    '''
    
    dfs = []
    
    models = [
          ('LogReg', LogisticRegression()), 
          ('RF', RandomForestClassifier()),
          ('KNN', KNeighborsClassifier()),
          ('SVM', SVC()), 
          ('GNB', GaussianNB()),
          ('XGB', XGBClassifier())
        ]
    
    results = []
    
    names = []
    
    scoring = ['accuracy', 'precision_weighted', 'recall_weighted', 'f1_weighted', 'roc_auc']
    
    target_names = ['win', 'loss']
    
    for name, model in models:
        
        kfold = model_selection.KFold(n_splits=5, shuffle=True, random_state=90210)
        cv_results = model_selection.cross_validate(model, X_train, y_train, cv=kfold, scoring=scoring)
        clf = model.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        
        print(name)
        print(classification_report(y_test, y_pred, target_names=target_names))
        
        results.append(cv_results)
        names.append(name)
        
        this_df = pd.DataFrame(cv_results)
        this_df['model'] = name
        dfs.append(this_df)
        
    final = pd.concat(dfs, ignore_index=True)
    
    return final
final = run_exps(X_train, y_train, X_test, y_test)
final

# Model Performance

In [ ]:
bootstraps = []
for model in list(set(final.model.values)):
    model_df = final.loc[final.model == model]
    bootstrap = model_df.sample(n=30, replace=True)
    bootstraps.append(bootstrap)
        
bootstrap_df = pd.concat(bootstraps, ignore_index=True)
results_long = pd.melt(bootstrap_df,id_vars=['model'],var_name='metrics', value_name='values')
time_metrics = ['fit_time','score_time'] # fit time metrics
## PERFORMANCE METRICS
results_long_nofit = results_long.loc[~results_long['metrics'].isin(time_metrics)] # get df without fit data
results_long_nofit = results_long_nofit.sort_values(by='values')
## TIME METRICS
results_long_fit = results_long.loc[results_long['metrics'].isin(time_metrics)] # df with fit data
results_long_fit = results_long_fit.sort_values(by='values')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(20, 12))
sns.set(font_scale=2.5)
g = sns.boxplot(x="model", y="values", hue="metrics", data=results_long_nofit, palette="Set3")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.title('Comparison of Model by Classification Metric')
plt.savefig('./benchmark_models_performance.png',dpi=300)

In [ ]:
plt.figure(figsize=(20, 12))
sns.set(font_scale=2.5)
g = sns.boxplot(x="model", y="values", hue="metrics", data=results_long_fit, palette="Set3")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.title('Comparison of Model by Fit and Score Time')
plt.savefig('./benchmark_models_time.png',dpi=300)

In [ ]:
metrics = list(set(results_long_nofit.metrics.values))
bootstrap_df.groupby(['model'])[metrics].agg([np.std, np.mean])

# Final Selected Model
- grid search for parameters 
- Gaussian NB

In [ ]:
# Gaussian NB only has one parameter 'var_smoothing'
# Portion of the largest variance of all features that is added to variances for calculation stability.
# Number of different combinations of parameters 

from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

nb_classifier = GaussianNB()

target_names = ['Win', 'Loss']

params_NB = {'var_smoothing': np.logspace(0,-9, num=100)}
kfold = model_selection.KFold(n_splits=5, shuffle=True, random_state=90210)

gs_NB = GridSearchCV(estimator=nb_classifier, 
                 param_grid=params_NB, 
                 cv=kfold,   
                 verbose=1, 
                 scoring='accuracy', n_jobs=-1) 

gs_NB.fit(X_train, y_train)

best_gs_grid = gs_NB.best_estimator_
best_gs_grid.fit(X_train, y_train)
y_pred_best_gs = best_gs_grid.predict(X_test)

print(classification_report(y_test, y_pred_best_gs, target_names=target_names))
gs_NB.best_params_

In [ ]:
from sklearn.metrics import confusion_matrix
confusionMatrix = confusion_matrix(y_test, y_pred_best_gs)
print(confusionMatrix)

# Confusion Matrix

In [ ]:
confusionMatrix = confusion_matrix(y_test, y_pred_best_gs)  

    # Code below prints model accuracy information
print('Coefficient Information:')

for i in range(len(featureColumns)):  

    logregCoefficients = logreg.coef_

    currentFeature = featureColumns[i]
    currentCoefficient = logregCoefficients[0][i]

    print(currentFeature + ': ' + str(currentCoefficient))

print('----------------------------------')

print("Accuracy:", metrics.accuracy_score(Y_test, Y_pred))
print("Precision:", metrics.precision_score(Y_test, Y_pred))
print("Recall:", metrics.recall_score(Y_test, Y_pred))

print('----------------------------------')

print('Confusion Matrix:')
print(confusionMatrix)

# Saving Model

In [ ]:
import pickle

# Saves the model in folder to be used in future
# filename should be end in '.pkl'
def save_model(model, filename):

    with open(filename, 'wb') as file:
        pickle.dump(model, file)
save_model()